# Prepare the data for the analysis

The data we are going to use for this course was generated using Synthea, a tool that generates synthetic patient data. From the data generated by Synthea, we extracted
the information for patients with prostate cancer only and copied only the relevant files into data_prostate/parquet. You can learn more about how this preparation step
was done in data_generation/reproduce_data_generation.txt. 

You can learn more about Synthea [in their webpage](https://github.com/synthetichealth/synthea). Of particular relevance for this section is the 
[data dictionary](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary) where there is a relation of files and the meaning of each
column on them, and the [prostate cancer module](https://synthetichealth.github.io/module-builder/#veteran_prostate_cancer) that explains the patient journey.

In this notebook, we are going to extract the data we need for our analysis and derive new variables as needed.
As a first step, we load the packages we need and define where the data is located

In [1]:
import os 

import pandas as pd 
import numpy as np

# where to find the data in parquet format
data_folder = "~/work/data/parquet"


## Get the diagnoses dates and disease progression.

All patients have an initial neoplasm diagnosis. After that, the patients are diagnosed with either carcinoma in situ (localized) or metastatic carcinoma (the cancer has spread to other locations). Later on, some patients with carcinoma in situ may progress to a metastatic carcinoma. Let's collect the dates for these diagnoses and the disease progression for each patient.

For this task,  we need to look at is "conditions". As you can see below, this file contains an identifier for each patient, the code and the description for the situation or diagnosis (disease), 
an identifier for the encounter (patient visiting the doctor) and a date for the start and stop of that encounter.


In [2]:
# we can read the data with the method read_parquet (as the data is stored in parquet format)
# os.path.join will simply concatenate the path to the data (data_folder) and the filename of the file we want to open
conditions = pd.read_parquet(os.path.join(data_folder, "conditions.parquet"))
conditions.head()

,start,stop,patient,encounter,code,description
0,1936-06-30,1936-06-30,57b843cd-26ad-f16a-6979-63c799ee22b0,47643225-48ea-157a-17bc-3bd602158518,314529007,Medication review due (situation)
1,1936-08-04,1936-08-04,57b843cd-26ad-f16a-6979-63c799ee22b0,db678344-fe1f-cddd-9319-8c230b384158,314529007,Medication review due (situation)
2,1936-10-06,1936-10-06,57b843cd-26ad-f16a-6979-63c799ee22b0,381d681f-7e27-05ab-1105-a776c024239e,314529007,Medication review due (situation)
3,1936-12-08,1936-12-08,57b843cd-26ad-f16a-6979-63c799ee22b0,5225513c-61b9-b4fd-dc45-fa689a35c63a,314529007,Medication review due (situation)
4,1937-03-09,1937-03-09,57b843cd-26ad-f16a-6979-63c799ee22b0,4eaf9629-b501-b10f-8892-93938b2c59bd,314529007,Medication review due (situation)


We would like to extract from this file the three relevant diagnosis for prostate cancer, togheter with the start date (date of diagnosis of the disease).

In [3]:
# Here we have the medical codes corresponding to the three diagnosis of interest
prostate_codes = [126906006, # Neoplasm of prostate
                92691004, # Carcinoma in situ of prostate (disorder)
                314994000, # Metastasis from malignant tumor of prostate (disorder)
                  ]
# We filter the conditions dataframe to keep only the rows corresponding to the prostate conditions
prostate_conditions = conditions.loc[conditions["code"].isin(prostate_codes), ["patient", "description", "start", ]]
# convert start to a pandas datetime for it to be easier to manipulate later. We introduce UTC timezone as other columns will be in UTC
prostate_conditions["start"] = pd.to_datetime(prostate_conditions["start"], utc=True)
prostate_conditions.head()

,patient,description,start
179,57b843cd-26ad-f16a-6979-63c799ee22b0,Neoplasm of prostate,2022-11-16 00:00:00+00:00
180,57b843cd-26ad-f16a-6979-63c799ee22b0,Carcinoma in situ of prostate (disorder),2022-11-16 00:00:00+00:00
335,14ef189d-dd30-b357-3eae-81baf7cbcfe2,Neoplasm of prostate,2016-02-18 00:00:00+00:00
336,14ef189d-dd30-b357-3eae-81baf7cbcfe2,Carcinoma in situ of prostate (disorder),2016-02-18 00:00:00+00:00
337,14ef189d-dd30-b357-3eae-81baf7cbcfe2,Metastasis from malignant tumor of prostate (d...,2016-05-18 00:00:00+00:00


As you see we have the data in long format, and what we would like to have is in wide format, i.e. one column for each diagnosis containing the date of the diagnosis.

In [4]:
# pivot to have one column for each of neoplasm, in situ, metastatic. We use the pivot method
prostate_piv = prostate_conditions.pivot(index="patient", values="start", columns="description")
# rename the columns to be more informative, use the rename method and pass a dictionary with the labels
col_names = {'Carcinoma in situ of prostate (disorder)': 'carcinoma_in_situ_dx_date',
             'Metastasis from malignant tumor of prostate (disorder)': 'metastasis_dx_date',
             'Neoplasm of prostate': 'neoplasm_dx_date'}
prostate_piv = prostate_piv.rename(columns=col_names)
prostate_piv.head()

description,carcinoma_in_situ_dx_date,metastasis_dx_date,neoplasm_dx_date
patient,,,
00b99a22-6749-4c13-b0f8-5d3957263377,2023-01-06 00:00:00+00:00,2023-04-06 00:00:00+00:00,2023-01-06 00:00:00+00:00
00c9fc77-8956-e9f2-9637-ec1384e5e4f0,2023-08-14 00:00:00+00:00,NaT,2023-08-14 00:00:00+00:00
00d0790e-25c5-25b8-b021-2a51dde140a0,2017-06-12 00:00:00+00:00,NaT,2017-06-12 00:00:00+00:00
00ea3106-3410-65aa-5e07-84ee326e3432,2018-09-15 00:00:00+00:00,NaT,2018-09-15 00:00:00+00:00
00f6f5e6-7f25-587e-1551-ee423dd0f22d,2018-01-07 00:00:00+00:00,NaT,2018-01-07 00:00:00+00:00


Let's finally build a variable to capture these progressions.

In [5]:
def assign_progression(df):
    """
    This functions gets a row of the dataframe and assigns a progression status
    based in the three diagnosis dates
    """
    # if there is metastatisis not carcinoma in situ
    if not pd.isna(df["metastasis_dx_date"]) and pd.isna(df["carcinoma_in_situ_dx_date"]):
        return "metastatic"
    # if there is carcinoma in situ but not metastasis
    elif not pd.isna(df["carcinoma_in_situ_dx_date"]) and pd.isna(df["metastasis_dx_date"]):
        return "in situ"
    # else it means it started with in situ and progressed to metastatic
    else:
        return "in situ -> metastatic"
# apply the function to the dataframe and save it in a variable "progression"
prostate_piv["progression"] = prostate_piv.apply(assign_progression, axis=1)
# let's see the counts
prostate_piv["progression"].value_counts()

progression
in situ                  752
in situ -> metastatic     94
metastatic                81
Name: count, dtype: int64

## Medications

Patients may receive one or two rounds of medication. We would like to know how many rounds of medications the patients got and the start of each round.

For this purpose we need the medications file. Here there are recorded all medications given to the patients for all purposes. We filter for medications given for
prostate cancer only and extract a few columns we need. 

We then take a look at what is the count of medications. As you can see there are two medications with exactly the same counts: Leuprolide Acetate (an hormonal supressing therapy) and Docetaxel (chemotherapy), and there are no other medications given to the patients. 

In [6]:
# read the medications file
medications = pd.read_parquet(os.path.join(data_folder, "medications.parquet"))
# filter the medications and keep only the ones related to prostate cancer
prostate_medications = medications.loc[medications["reasondescription"]=="Neoplasm of prostate", ["patient", "description", "start"]]
prostate_medications.groupby("description").size()

description
0.25 ML Leuprolide Acetate 30 MG/ML Prefilled Syringe    1536
1 ML DOCEtaxel 20 MG/ML Injection                        1536
dtype: int64

Taking a look to the data, we notice that these two drugs are given at exactly the same time. 

In [7]:
prostate_medications.head(6)

,patient,description,start
521,57b843cd-26ad-f16a-6979-63c799ee22b0,1 ML DOCEtaxel 20 MG/ML Injection,2022-11-16 00:05:41+00:00
522,57b843cd-26ad-f16a-6979-63c799ee22b0,0.25 ML Leuprolide Acetate 30 MG/ML Prefilled ...,2022-11-16 00:05:41+00:00
531,57b843cd-26ad-f16a-6979-63c799ee22b0,1 ML DOCEtaxel 20 MG/ML Injection,2023-02-14 04:36:41+00:00
532,57b843cd-26ad-f16a-6979-63c799ee22b0,0.25 ML Leuprolide Acetate 30 MG/ML Prefilled ...,2023-02-14 04:36:41+00:00
931,14ef189d-dd30-b357-3eae-81baf7cbcfe2,1 ML DOCEtaxel 20 MG/ML Injection,2016-02-18 18:54:01+00:00
932,14ef189d-dd30-b357-3eae-81baf7cbcfe2,0.25 ML Leuprolide Acetate 30 MG/ML Prefilled ...,2016-02-18 18:54:01+00:00


As the two drugs are given always in combination, we will consider them as one treatment and we will keep the date for only one of them, and then calculate the dates for the different rounds of medications, which we will call "lines of treatment". We can see that patients may have one or two lines of treatment.

In [8]:
# get a single line per patient and the start date of the line and sort by start date
lines = prostate_medications[["patient", "start"]].drop_duplicates().sort_values(by="start")
# create a variable with the number of the medication round (medication line)
lines['line_number'] = lines.groupby('patient').cumcount()+1
# pivot by the medication line
lines_piv = lines.pivot(index="patient", columns="line_number", values="start").reset_index()
# rename the columns to be more informative
lines_piv = lines_piv.rename(columns={1: "line_1_start", 2: "line_2_start",})
# build a column indication werther the patient received one or two lines of treatment
lines_piv["lines_of_treatment"] = np.where(pd.isna(lines_piv["line_2_start"]), "L1", "L2")
lines_piv.head()

line_number,patient,line_1_start,line_2_start,lines_of_treatment
0,00b99a22-6749-4c13-b0f8-5d3957263377,2023-01-06 08:25:13+00:00,NaT,L1
1,00c9fc77-8956-e9f2-9637-ec1384e5e4f0,2023-08-14 06:51:32+00:00,2023-11-12 11:20:32+00:00,L2
2,00d0790e-25c5-25b8-b021-2a51dde140a0,2017-06-12 09:40:53+00:00,2017-09-10 14:33:53+00:00,L2
3,00ea3106-3410-65aa-5e07-84ee326e3432,2018-09-15 19:29:26+00:00,2018-12-15 00:56:26+00:00,L2
4,00f6f5e6-7f25-587e-1551-ee423dd0f22d,2018-01-07 00:52:36+00:00,2018-04-07 05:18:36+00:00,L2


## Demographics

We next get demographics patient data from the patients file.

In [9]:
demographics = pd.read_parquet(os.path.join(data_folder, "patients.parquet")).rename(columns={'id': 'patient'})
# keep only the columns we need later
cols_of_interest = ["patient", "deathdate", "birthdate", "race", "ethnicity", "gender", "income", ]
demographics = demographics.loc[:, cols_of_interest]
# transform deathdate and birthdate to datetime to later be able to do computations with them easily
demographics["deathdate"] = pd.to_datetime(demographics["deathdate"], utc=True)
demographics["birthdate"] = pd.to_datetime(demographics["birthdate"], utc=True)

demographics.head()

,patient,deathdate,birthdate,race,ethnicity,gender,income
0,57b843cd-26ad-f16a-6979-63c799ee22b0,NaT,1936-06-30 00:00:00+00:00,white,nonhispanic,M,11515
1,14ef189d-dd30-b357-3eae-81baf7cbcfe2,2017-12-11 00:00:00+00:00,1951-06-14 00:00:00+00:00,white,nonhispanic,M,67593
2,6c9d7683-f67f-2893-5776-6cbe1586081c,NaT,1950-05-09 00:00:00+00:00,white,nonhispanic,M,40570
3,e87004f8-11f9-37b5-ce3e-4b95bb022829,NaT,1953-08-06 00:00:00+00:00,white,nonhispanic,M,18639
4,8e1cd266-fa31-1f98-f6fd-2fbafd293bdd,NaT,1955-08-27 00:00:00+00:00,white,hispanic,M,87359


Let's look at the race variable. There are very few individuals from the races asian, hawaiian, other and native.

In [10]:
demographics.race.value_counts()

race
white       782
black        79
asian        40
hawaiian     13
other         9
native        4
Name: count, dtype: int64

Therefore we are going to merge them togheter into the category "other" 

In [11]:
race_map = {'white': 'white', 'black':'black', 'asian':'other',
            'hawaiian':'other', 'other':'other', 'native':'other'}
demographics['race'] = demographics['race'].map(race_map)
demographics.race.value_counts()

race
white    782
black     79
other     66
Name: count, dtype: int64

We can also observe that the most frequent race is "white". Later, when we do the Cox regression analysis, we would like that the most frequent race becomes the denominator for the analysis. In order to achieve that, we will transfom the race variable into an ordered categorical variable.

In [12]:
demographics['race'] = pd.Categorical(demographics['race'], categories=["white", "black", "other", ], ordered=True)

Let's take a look at the variable ethnicity.

In [13]:
demographics.ethnicity.value_counts()

ethnicity
nonhispanic    830
hispanic        97
Name: count, dtype: int64

Again, we would like that the most frequent ethnicity becomes the denominator, therefore we transform the variable to an ordered categorical.

In [14]:
demographics['ethnicity'] = pd.Categorical(demographics['ethnicity'], categories=["nonhispanic", "hispanic", ], ordered=True)

## Last patient activity

We need to determine when was the last known contact with the patient. We will use the encounters, procedures and conditions files, looking on each what is the latest record for each patient, and finally merging them and taking the latest date among them for each patient. That is the last know activity for the patient. We generate this variable and we will take a look on what is the 
contribution of each file to it, just for informative purposes (it is not relevant later in the analysis the source of the information). 

In [15]:
# read the encounters file
encounters = pd.read_parquet(os.path.join(data_folder, "encounters.parquet"))
# sort by the start date descending and get the first (latest) encounter for each patient
last_encounter = encounters[["patient", "start", "description"]].sort_values(by="start", ascending=False).groupby("patient").first()
# rename the columns to be more informative
last_encounter = last_encounter.rename(columns={"start": "last_activity", "description": "description"})
# let's keep that the source of the data is the encounter, just for our records
last_encounter['source'] = "encounter"

# repeat with procedures and conditions
procedures = pd.read_parquet(os.path.join(data_folder, "procedures.parquet"))
last_procedure = procedures[["patient", "stop", "description"]].sort_values(by="stop", ascending=False).groupby("patient").first()
last_procedure = last_procedure.rename(columns={"stop": "last_activity", "description": "description"})
last_procedure['source'] = "procedure"

conditions = pd.read_parquet(os.path.join(data_folder, "conditions.parquet"))
last_condition = conditions[["patient", "start", "description"]].sort_values("start", ascending=False).groupby("patient").first()
last_condition = last_condition.rename(columns={"start": "last_activity", "description": "description"})
last_condition['last_activity'] = pd.to_datetime(last_condition["last_activity"]).dt.tz_localize("UTC")#.dt.replace_time_zone("UTC")
last_condition['source'] = "condition"

# now we concatenate the three dataframes
all_events = pd.concat([
  last_encounter,
  last_condition,
  last_procedure,
])
# sort the values by date in descending order and group by patient to get the latest event for each patient
last_event = all_events.sort_values(by=["last_activity", "source"], ascending=False).groupby("patient").first().reset_index()
# let's see the counts for each source, just for our records
last_event.groupby('source').size()

source
condition      1
encounter    400
procedure    526
dtype: int64

# Reason for Death

We also would like to know the reason for death for the patients, when available. The information is contained in the encounters file. Let's extract the data and look at the reasons for death.

In [16]:
# let's look for the Death certificate in the encounters and get the reason description, rename to death_reason
deathcert = encounters.loc[encounters["description"]=="Death Certification", ["patient", "reasondescription"]].rename(columns={"reasondescription": "death_reason"})
# let's check the counts for the different reasons
deathcert.death_reason.value_counts()

death_reason
Neoplasm of prostate                                               152
End-stage renal disease (disorder)                                  29
Non-small cell lung cancer (disorder)                               26
Acute ST segment elevation myocardial infarction (disorder)         11
Chronic congestive heart failure (disorder)                          7
COVID-19                                                             5
Chronic obstructive bronchitis (disorder)                            4
Pneumonia                                                            4
Alzheimer's disease (disorder)                                       4
Pulmonary emphysema (disorder)                                       4
Small cell carcinoma of lung (disorder)                              4
Acute non-ST segment elevation myocardial infarction (disorder)      4
Overlapping malignant neoplasm of colon                              3
Ischemic heart disease (disorder)                               

As you can see there are many possible reasons for death, but we are interested here on those patients dying from prostate cancer vs other causes. Therefore, let's categorize the reasons on these to categories.

In [17]:
# recode the death reasons to have only two categories
deathcert["death_reason"] = np.where(deathcert["death_reason"]=="Neoplasm of prostate", "prostate cancer", "Other causes")
deathcert["death_reason"].value_counts()

death_reason
prostate cancer    152
Other causes       110
Name: count, dtype: int64

## Followup

For our survival analysis we need to determine the followup time for each patient. We first have to calculate what was the latest followup date, as either the date of death if present or the latest known activity we calculated before. Then we can calculate the followup time as the difference between the lastest followup date and the date of the diagnosis of the disease. We will calculate two different followup times, one with the neoplasm diagnosis, present for all patients, and another with the metastasis diagnosis, which is not present for all patients.

In [18]:
# merge prostate_piv with the last event and demographics, to get the date of death
prostate_df = (prostate_piv.merge(last_event.loc[:, ["patient", "last_activity"]], on="patient", how="left")
                            .merge(demographics, on="patient", how="left"))
# calculate the last followup date: if there is no death date, use the last activity date, otherwise death date
prostate_df["last_followup"] = np.where(pd.isna(prostate_df["deathdate"]), prostate_df["last_activity"], prostate_df["deathdate"])
# now calculate the followup time in months from the neoplasm and metastasis diagnosis
prostate_df["followup_neoplasm_months"] = (prostate_df["last_followup"] - prostate_df["neoplasm_dx_date"]).dt.days/30.4375
prostate_df["followup_metastasis_months"] = (prostate_df["last_followup"] - prostate_df["metastasis_dx_date"]).dt.days/30.4375
prostate_df[['patient', 'neoplasm_dx_date', 'metastasis_dx_date', 'last_followup', 'followup_neoplasm_months', 'followup_metastasis_months']].head()

,patient,neoplasm_dx_date,metastasis_dx_date,last_followup,followup_neoplasm_months,followup_metastasis_months
0,00b99a22-6749-4c13-b0f8-5d3957263377,2023-01-06 00:00:00+00:00,2023-04-06 00:00:00+00:00,2024-01-13 07:51:43+00:00,12.221766,9.264887
1,00c9fc77-8956-e9f2-9637-ec1384e5e4f0,2023-08-14 00:00:00+00:00,NaT,2024-07-08 08:26:15+00:00,10.809035,NaN
2,00d0790e-25c5-25b8-b021-2a51dde140a0,2017-06-12 00:00:00+00:00,NaT,2024-07-22 11:27:51+00:00,85.322382,NaN
3,00ea3106-3410-65aa-5e07-84ee326e3432,2018-09-15 00:00:00+00:00,NaT,2024-06-22 20:26:59+00:00,69.223819,NaN
4,00f6f5e6-7f25-587e-1551-ee423dd0f22d,2018-01-07 00:00:00+00:00,NaT,2018-10-15 00:00:00+00:00,9.232033,NaN


## Calculate more variables

We need to calculate a few more variables:
* The age at the neoplasm diagnosis date (the diagnosis year minus the birth year)
* We will divide the income by 1000 to make it more readable.
* A variable to indicate if the patient has died or not.

In [19]:
# age at neoplasm diagnosis
prostate_df["age_at_dx"] = prostate_df["neoplasm_dx_date"].dt.year - prostate_df["birthdate"].dt.year
# divide income by 1000 to make it more readable
prostate_df["income"] = prostate_df["income"]/1000
# make a variable to indicate if the patient died
prostate_df["is_dead"] = np.where(pd.isna(prostate_df["deathdate"]), 0, 1)
prostate_df[['patient', 'age_at_dx', 'income', 'is_dead']].head()

,patient,age_at_dx,income,is_dead
0,00b99a22-6749-4c13-b0f8-5d3957263377,67,50.249,0
1,00c9fc77-8956-e9f2-9637-ec1384e5e4f0,63,126.045,0
2,00d0790e-25c5-25b8-b021-2a51dde140a0,60,167.457,0
3,00ea3106-3410-65aa-5e07-84ee326e3432,65,94.640,0
4,00f6f5e6-7f25-587e-1551-ee423dd0f22d,69,323.357,1


## Join all the data

Finally, we will join all the data we generated before into a single dataframe, bringing in only those columns we need for further analysis. We also correct the reason for death assigning either Alive or Missing as categories depeding if the patient is still alive or the reason for death is missing. Let's take a look at the final dataset.

In [20]:
# from prostate_df let's keep only the variables we need
# then merge with the other dataframes we need data from, also taking only the columns we need for further analysis
prostate_df = (prostate_df[["patient", "progression", "race", "ethnicity", "gender", "income", "age_at_dx", "followup_neoplasm_months", "followup_metastasis_months", "is_dead"]]
                .merge(lines_piv.loc[:, ["patient", "lines_of_treatment"]], on="patient", how="left")
                .merge(deathcert, on="patient", how="left")
)
# if no death reaason, check if the patient is alive, if so, assign "alive" otherwise "missing"
prostate_df.loc[pd.isna(prostate_df["death_reason"]), "death_reason"] = np.where(prostate_df.loc[pd.isna(prostate_df["death_reason"]), "is_dead"]==0, "Alive", "Missing")
prostate_df

,patient,progression,race,ethnicity,gender,income,age_at_dx,followup_neoplasm_months,followup_metastasis_months,is_dead,lines_of_treatment,death_reason
0,00b99a22-6749-4c13-b0f8-5d3957263377,in situ -> metastatic,white,nonhispanic,M,50.249,67,12.221766,9.264887,0,L1,Alive
1,00c9fc77-8956-e9f2-9637-ec1384e5e4f0,in situ,white,nonhispanic,M,126.045,63,10.809035,NaN,0,L2,Alive
2,00d0790e-25c5-25b8-b021-2a51dde140a0,in situ,white,nonhispanic,M,167.457,60,85.322382,NaN,0,L2,Alive
3,00ea3106-3410-65aa-5e07-84ee326e3432,in situ,other,nonhispanic,M,94.640,65,69.223819,NaN,0,L2,Alive
4,00f6f5e6-7f25-587e-1551-ee423dd0f22d,in situ,white,nonhispanic,M,323.357,69,9.232033,NaN,1,L2,Other causes
...,...,...,...,...,...,...,...,...,...,...,...,...
922,fe62a0ec-c6ef-0263-2f27-16cf717b77e2,in situ,white,nonhispanic,M,35.010,63,68.501027,NaN,0,L2,Alive
923,fe79f80d-341b-9f1b-9a97-065463a48329,in situ,white,nonhispanic,M,49.401,66,60.583162,NaN,1,L2,Other causes
924,ff0b9c41-268f-a33b-859e-6ad759708d79,in situ,white,nonhispanic,M,18.091,66,45.995893,NaN,0,L2,Alive
925,ff0cea11-dfe1-34f7-ee5d-d75622e5198e,in situ,white,nonhispanic,M,38.561,66,12.188912,NaN,0,L2,Alive


## Save the data

Finally, let's save the data into a parquet file for further analysis.

In [21]:
# create the folder to store the data
os.makedirs('/home/jovyan/work/data/ard', exist_ok=True)
# write the data to parquet format
prostate_df.to_parquet("/home/jovyan/work/data/ard/patients.parquet")